# Projeto Final de Avaliação Substitutiva

### Proposta: “Avaliação de métodos de reconhecimento facial utilizando visão computacional”

In [1]:
#bibliotecas
import cv2
import os
import numpy as np
import time

#### Variáveis

In [2]:
#Nome dos indivídios
subjects = ["", "Willian Bonner", "Tiago Leifert", "Cesar Tralli" ]


#Estrutura de Pastas
FACE_DETECTOR_MODEL = '../model/lbpcascade_frontalface.xml'
PATH_TRAINING_DATA  = '../dataset/training'
PATH_RESULT_DATA    = '../dataset/result'
PATH_TEST_DATA      = '../dataset/test'


#Redimensionamento das Imagens
IMAGE_WIDTH         = 500
IMAGE_HEIGTH        = 400


#Modelo de Reconhecimento
recognizer_eigen  = cv2.face.EigenFaceRecognizer_create()
recognizer_fisher = cv2.face.FisherFaceRecognizer_create()
recognizer_lbph   = cv2.face.LBPHFaceRecognizer_create()

print( "EigenFaceRecognizer [", recognizer_eigen , "]" )
print( "FisherFaceRecognizer[", recognizer_fisher, "]" )
print( "LBPHFaceRecognizer  [", recognizer_lbph,   "]" )

EigenFaceRecognizer [ <face_EigenFaceRecognizer 0000019DD9FA4110> ]
FisherFaceRecognizer[ <face_FisherFaceRecognizer 0000019DD2944EB0> ]
LBPHFaceRecognizer  [ <face_LBPHFaceRecognizer 0000019DD2944ED0> ]


#### Detecção das faces e normalizando-as em escala de cinza

In [3]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier( FACE_DETECTOR_MODEL )

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    if (len(faces) == 0):
        return None, None
    
    (x, y, w, h) = faces[0]
    
    return gray[y:y+w, x:x+h], faces[0]

#### Preparando os dados para treinamento (captura das faces)

In [4]:
def prepare_training_data(data_folder_path):
    
    dirs = os.listdir(data_folder_path)
    
    faces = []
    labels = []
    
    for dir_name in dirs:

        cv2.destroyAllWindows()
        
        label = int( dir_name )
        
        subject_dir_path = data_folder_path + "/" + dir_name
    
        subject_images_names = os.listdir(subject_dir_path)
        
        for image_name in subject_images_names:
            
            if image_name.startswith("."):
                continue;
            
            image_path = subject_dir_path + "/" + image_name

            image = cv2.imread(image_path)
            image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGTH) )

            cv2.imshow( ( "Treinando - " + subjects[label] ), cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGTH)))
            
            subjects
            cv2.waitKey(100)
            
            face, rect = detect_face(image)
            
            if face is not None:
                
                face = cv2.resize( face, (IMAGE_WIDTH, IMAGE_HEIGTH))
                
                faces.append( face )
                labels.append( label )
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    
    
    return faces, labels

In [5]:
faces, labels = prepare_training_data( PATH_TRAINING_DATA )
print("Total faces [", len(faces),  ']' )
print("Total labels[", len(labels), ']' )

Total faces [ 79 ]
Total labels[ 79 ]


In [6]:
#Treinamento
recognizer_eigen.train(faces, np.array(labels))
recognizer_fisher.train(faces, np.array(labels))
recognizer_lbph.train(faces, np.array(labels))

In [7]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)


In [8]:
def predict(test_img, face_recognizer):
    
    img = test_img.copy()
    face, rect = detect_face(img)
    
    face = cv2.resize( face, (IMAGE_WIDTH, IMAGE_HEIGTH))
    
    label, confidence = face_recognizer.predict(face)
    label_text = subjects[label]
    
    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1]-5)

    return img, label_text, confidence

In [9]:
def save_image_result( directory, image, image_name, people_name, accuracy ):
    
    try:
        image_result = image.copy()

        cv2.putText( image_result, people_name,(50,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0,0,255), 2, cv2.LINE_AA )
        cv2.putText( image_result, str( accuracy ),(100,100), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (255,255,255), 2, cv2.LINE_AA )

        cv2.imwrite( directory + "/" + image_name , image_result )
    except:
        print("Erro em salvar a imagem")

In [10]:
#Testes
images_test = os.listdir( PATH_TEST_DATA )
        
for image_name in images_test:
    img_test = cv2.imread( PATH_TEST_DATA + "/" + image_name )

    img_test = cv2.resize( img_test, (IMAGE_WIDTH, IMAGE_HEIGTH) )
    
    try:
        ( predicted, name, accuracy ) = predict( img_test, recognizer_lbph )
        save_image_result( PATH_RESULT_DATA + "/" + "LBPHFaceRecognizer", img_test, image_name, name, accuracy )
    except Exception as e:
        print("Erro recognizer_lbph", e )
    
    try:
        ( predicted, name, accuracy ) = predict( img_test, recognizer_eigen )
        save_image_result( PATH_RESULT_DATA + "/" + "EigenFaceRecognizer", img_test, image_name, name, accuracy )
    except Exception as e:
        print("Erro recognizer_eigen", e)
    
    try:
        ( predicted, name, accuracy ) = predict( img_test, recognizer_fisher )
        save_image_result( PATH_RESULT_DATA + "/" + "FisherFaceRecognizer", img_test, image_name, name, accuracy )
    except Exception as e:
        print("Erro recognizer_fisher", e)

Erro recognizer_lbph OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Erro recognizer_eigen OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Erro recognizer_fisher OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Erro recognizer_lbph OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Erro recognizer_eigen OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Erro recognizer_fisher OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: erro

Erro recognizer_eigen OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Erro recognizer_fisher OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

